<a href="https://colab.research.google.com/github/IlyaZutler/Project-3-Berlin-Airbnb-Ratings/blob/main/1%20Data%20Preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data Preparation 2**

Dataset contains 450 thousand comments of 23.5 thousand unique apartments.

We will analyze comments and store aggregated data for apartments.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

!pip install openpyxl -q
import openpyxl
import pickle

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Comment analisis
!pip install googletrans==4.0.0-rc1 -q
!pip install langdetect googletrans==4.0.0-rc1 -q
!pip install --upgrade googletrans==3.1.0a0 -q # Upgrade googletrans to a stable version.

from textblob import TextBlob
from langdetect import detect
from googletrans import Translator

import time
from datetime import datetime

# Set display options
%matplotlib inline
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_colwidth', 1000)

In [45]:
apartments = pd.read_excel('https://raw.githubusercontent.com/IlyaZutler/Project-3-Berlin-Airbnb-Ratings/main/apartments.xlsx', engine='openpyxl')

In [27]:
comments = pd.read_excel('https://raw.githubusercontent.com/IlyaZutler/Project-3-Berlin-Airbnb-Ratings/main/Comments_full.xlsx', engine='openpyxl')

## Comment Analysis



In [ ]:
def analyze_sentiment(text):
    try:
        # Определяем язык текста
        lang = detect(text)
    except:
        lang = 'en'  # Если не удалось определить язык, предполагаем английский

    # Если язык не английский, переводим текст
    if lang != 'en':
        translator = Translator()
        translator.raise_Exception = True # Add this line to enable exception raising
        try:
            translated = translator.translate(text, src=lang, dest='en')
            text = translated.text
        except Exception as e:
            print(f"Translation error: {e}") # Print the error message
            return 0 # Return a neutral sentiment if translation fails

    # Анализ настроения
    blob = TextBlob(text)
    sentiment = blob.sentiment
    return sentiment[0]

comments['polarity'] = comments['Comments'].apply(analyze_sentiment)

# Применяем анализ настроения ко всем комментариям
# Introduce a delay to avoid overwhelming the API
# for i, comment in enumerate(comments['Comments']):
#     comments['polarity'][i] = analyze_sentiment(comment)
#     time.sleep(1) # Wait for one second before processing the next comment

In [ ]:
# import pandas as pd
# !pip install langdetect -q
# from langdetect import detect
# from langdetect.lang_detect_exception import LangDetectException

# # Функция для попытки определения языка
# def detect_language(text):
#     try:
#         # Пытаемся определить язык
#         lang = detect(text)
#         return True  # Успешно определён
#     except LangDetectException:
#         return False  # Не удалось определить язык

# # Применяем функцию ко всему столбцу 'Text' и создаем новый столбец 'is_det'
# comments0['is_det'] = comments['Comments'].apply(detect_language)

In [40]:
#most Zero comments - wrong decoding
comments_noZ = comments[comments['polarity']!=0]
comments_noZ.shape

(431072, 3)

In [43]:
comments_grouped = comments_noZ.groupby('Listing ID').agg(
    Polarity_min=('polarity', 'min'),
    Polarity_max=('polarity', 'max'),
    Polarity_median=('polarity', 'median'),
    Polarity_mean=('polarity', 'mean')
)
comments_grouped.head()

,Polarity_min,Polarity_max,Polarity_median,Polarity_mean
Listing ID,,,,
2695,0.270000,0.776667,0.464583,0.477044
3176,0.085556,0.940000,0.427083,0.438078
7071,0.072000,1.000000,0.417121,0.451573
9991,0.420631,1.000000,0.459722,0.562904
14325,0.068182,1.000000,0.483929,0.470239


In [46]:
apartments = apartments.merge(comments_grouped, left_on='Listing ID', right_on='Listing ID', how='left')

## Preprocessing data

### Make time format

In [47]:
# from datetime import datetime

apartments['Host Since'] = pd.to_datetime(apartments['Host Since'])
apartments['First Review'] = pd.to_datetime(apartments['First Review'])
apartments['Last Review'] = pd.to_datetime(apartments['Last Review'])

today = pd.to_datetime(datetime.today().strftime('%Y-%m-%d'))

apartments['Host Since Years'] = (today - apartments['Host Since']).dt.days / 365.25
apartments['Last Review Years'] = (today - apartments['Last Review']).dt.days / 365.25
apartments['First Review Years'] = (today - apartments['First Review']).dt.days / 365.25

### Separating numeric and non-numeric data

In [48]:
apartments['Listing ID'] = apartments['Listing ID'].astype('category')
apartments['Postal Code'] = apartments['Postal Code'].astype('category')

In [49]:
# import pickle

with open('apartments2.pkl', 'wb') as f:
    pickle.dump(apartments, f)

In [ ]:
# apartments.to_excel('apartments_2.xlsx', index=False)

-------------------end of 1-----------------------